# 🔬 HBV 그룹 비교 분석 (Real Data)

**목적**: HBV Stage별 면역세포 분포 비교 (GSE182159 실제 데이터)

---

## 📊 HBV Stages

| Stage | 설명 |
|-------|------|
| IT | Immune Tolerant (면역 관용기) |
| IA | Immune Active (면역 활성기) |
| IC | Immune Control (면역 조절기) |
| GZ | Gray Zone |
| Resolved | 완치 |

## 📋 분석 단계

| STEP | 내용 |
|------|------|
| 1 | 환경 설정 및 데이터 로드 |
| 2 | 그룹별 세포 분포 비교 |
| 3 | 세포 유형별 비율 분석 |
| 4 | 조직별 (Blood vs Liver) 분석 |
| 5 | 탈진 마커 발현 분석 |
| 6 | 통계적 검정 |
| 7 | 결과 저장 |

⚠️ **사전 조건**: `Step1_HBV_real.ipynb` 실행 완료

In [ ]:
# @title STEP 1: 환경 설정 및 데이터 로드
import os, sys
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
ANALYSIS_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/comparison_results'
os.makedirs(OUTPUT_PATH, exist_ok=True)

# 패키지 설치
!pip install scanpy anndata scipy statsmodels seaborn mygene -q

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import scipy.sparse as sp
import gc

# Step 1 결과 로드 (전체 243K 분석 결과)
print('📊 분석 결과 검색 중...')
h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) if f.startswith('hbv_real_tahoe_analysis_full')]

if not h5ad_files:
    # full 버전이 없으면 다른 파일 찾기
    h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) if f.endswith('.h5ad')]

print(f'   발견된 파일: {h5ad_files}')
latest_file = sorted(h5ad_files)[-1]
print(f'   로드할 파일: {latest_file}')

print('\n📊 데이터 로딩 중 (대용량 파일)...')
adata = sc.read_h5ad(f'{ANALYSIS_PATH}/{latest_file}')

print(f'\n✅ STEP 1 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')

# HBV Stage 컬럼 확인
stage_col = None
for col in ['hbv_stage', 'Stage', 'stage']:
    if col in adata.obs.columns:
        stage_col = col
        break

if stage_col:
    print(f'\n📊 HBV Stage 분포 ({stage_col}):')
    print(adata.obs[stage_col].value_counts())
else:
    print('⚠️ HBV Stage 컬럼을 찾을 수 없습니다.')
    print(f'   사용 가능한 컬럼: {list(adata.obs.columns)}')

In [ ]:
# @title STEP 2: 그룹별 세포 분포 비교 (UMAP)
print('📊 그룹별 UMAP 시각화')

# HBV Stage 컬럼 확인
stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'

if stage_col not in adata.obs.columns:
    raise ValueError(f'⚠️ {stage_col} 컬럼이 없습니다!')

# 전체 UMAP (HBV Stage 색상)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sc.pl.umap(adata, color='cell_type', ax=axes[0], show=False,
           title='Cell Types', legend_loc='on data', legend_fontsize=8)

sc.pl.umap(adata, color=stage_col, ax=axes[1], show=False,
           title='HBV Stage')

sc.pl.umap(adata, color='tissue', ax=axes[2], show=False,
           title='Tissue (Blood vs Liver)')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step2_umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

# Stage별 개별 UMAP
stages = adata.obs[stage_col].dropna().unique()
n_stages = len(stages)
print(f'\n📊 Stage별 UMAP ({n_stages}개 그룹)')

cols = min(4, n_stages)
rows = (n_stages + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
axes = axes.flatten() if n_stages > 1 else [axes]

for i, stage in enumerate(sorted(stages)):
    if i < len(axes):
        adata_subset = adata[adata.obs[stage_col] == stage]
        sc.pl.umap(adata_subset, color='cell_type', ax=axes[i], show=False,
                   title=f'{stage} (n={len(adata_subset):,})', legend_loc='none')

# 빈 axes 숨기기
for j in range(i+1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step2_umap_by_stage.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 2 완료!')

In [ ]:
# @title STEP 3: 세포 유형별 비율 분석
print('📊 세포 유형별 비율 분석')

stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'

# 그룹별 세포 유형 비율 계산
cell_type_counts = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
cell_type_ratios = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0) * 100

print('\n📋 그룹별 세포 유형 비율 (%):')
print(cell_type_ratios.round(1))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Stacked bar
cell_type_ratios.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab10')
axes[0].set_title('Cell Type Distribution by HBV Stage', fontsize=12)
axes[0].set_xlabel('HBV Stage')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
axes[0].tick_params(axis='x', rotation=45)

# 2. Heatmap
sns.heatmap(cell_type_ratios.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1],
            cbar_kws={'label': 'Percentage (%)'})
axes[1].set_title('Cell Type Percentage Heatmap', fontsize=12)
axes[1].set_xlabel('HBV Stage')
axes[1].set_ylabel('Cell Type')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step3_cell_type_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# CSV 저장
cell_type_ratios.to_csv(f'{OUTPUT_PATH}/cell_type_ratios_by_stage.csv')

print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 조직별 분석 (Blood vs Liver)
print('📊 조직별 세포 분포 분석 (Blood vs Liver)')

stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'

# 조직별 세포 유형 분포
for tissue in ['Blood', 'Liver']:
    adata_tissue = adata[adata.obs['tissue'] == tissue]
    print(f'\n📊 {tissue} ({adata_tissue.n_obs:,} cells):')
    
    tissue_counts = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    tissue_ratios = tissue_counts.div(tissue_counts.sum(axis=1), axis=0) * 100
    print(tissue_ratios.round(1))

# 시각화: Blood vs Liver 비교
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, tissue in enumerate(['Blood', 'Liver']):
    adata_tissue = adata[adata.obs['tissue'] == tissue]
    tissue_counts = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    tissue_ratios = tissue_counts.div(tissue_counts.sum(axis=1), axis=0) * 100
    
    tissue_ratios.plot(kind='bar', stacked=True, ax=axes[idx], colormap='tab10')
    axes[idx].set_title(f'{tissue} - Cell Type by HBV Stage', fontsize=12)
    axes[idx].set_xlabel('HBV Stage')
    axes[idx].set_ylabel('Percentage (%)')
    axes[idx].tick_params(axis='x', rotation=45)
    if idx == 1:
        axes[idx].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
    else:
        axes[idx].legend().set_visible(False)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/step4_tissue_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 4 완료!')

In [ ]:
# @title STEP 5: 탈진 마커 발현 분석 (Gene Symbol → Ensembl 변환)
import mygene

print('🔬 T세포 탈진 마커 분석')

stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'

# 탈진/면역 관련 마커 (Gene Symbol)
markers_symbol = {
    'Exhaustion': ['PDCD1', 'HAVCR2', 'LAG3', 'CTLA4', 'TIGIT'],
    'Treg': ['FOXP3', 'IL2RA', 'IKZF2'],
    'Cytotoxic': ['GZMB', 'PRF1', 'IFNG', 'GNLY'],
    'Activation': ['CD69', 'HLA-DRA', 'CD38']
}

# Gene Symbol → Ensembl ID 변환
all_markers = []
for genes in markers_symbol.values():
    all_markers.extend(genes)

mg = mygene.MyGeneInfo()
results = mg.querymany(all_markers, scopes='symbol', fields='ensembl.gene', species='human', verbose=False)

symbol_to_ensembl = {}
for r in results:
    if 'ensembl' in r:
        ensembl = r['ensembl']
        if isinstance(ensembl, list):
            ensembl = ensembl[0]
        if isinstance(ensembl, dict) and 'gene' in ensembl:
            symbol_to_ensembl[r['query']] = ensembl['gene']

print(f'   Gene Symbol → Ensembl 변환: {len(symbol_to_ensembl)}/{len(all_markers)}')

# 데이터에서 사용 가능한 마커 확인
available_markers = {}
for category, genes in markers_symbol.items():
    ensembl_genes = [symbol_to_ensembl.get(g) for g in genes if symbol_to_ensembl.get(g) in adata.var_names]
    if ensembl_genes:
        available_markers[category] = {
            'ensembl': ensembl_genes,
            'symbol': [g for g in genes if symbol_to_ensembl.get(g) in adata.var_names]
        }
        print(f'   {category}: {available_markers[category]["symbol"]}')

# 탈진 마커 분석
if 'Exhaustion' in available_markers:
    print('\n📊 탈진 마커 Stage별 발현')
    
    n_markers = len(available_markers['Exhaustion']['ensembl'])
    fig, axes = plt.subplots(1, n_markers, figsize=(4*n_markers, 5))
    if n_markers == 1:
        axes = [axes]
    
    for ax, (ensembl, symbol) in zip(axes, zip(available_markers['Exhaustion']['ensembl'], 
                                              available_markers['Exhaustion']['symbol'])):
        sc.pl.violin(adata, ensembl, groupby=stage_col, ax=ax, show=False)
        ax.set_title(f'{symbol}')
        ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_PATH}/step5_exhaustion_markers.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print('⚠️ 탈진 마커를 찾을 수 없습니다.')

print(f'\n✅ STEP 5 완료!')

In [ ]:
# @title STEP 6: 통계적 검정
print('📊 통계적 유의성 검정')

stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'

# 1. Chi-square test (세포 유형 분포)
print('\n1️⃣ Chi-square test (세포 유형 × HBV Stage):')
contingency_table = adata.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
print(f'   Chi-square: {chi2:.2f}')
print(f'   Degrees of freedom: {dof}')
print(f'   p-value: {p_value:.2e}')
print(f'   결과: {"✅ 유의함 (p < 0.05)" if p_value < 0.05 else "❌ 유의하지 않음"}')

# 2. 조직별 Chi-square
print('\n2️⃣ 조직별 Chi-square test:')
for tissue in ['Blood', 'Liver']:
    adata_tissue = adata[adata.obs['tissue'] == tissue]
    cont_table = adata_tissue.obs.groupby([stage_col, 'cell_type']).size().unstack(fill_value=0)
    chi2_t, p_t, _, _ = stats.chi2_contingency(cont_table)
    print(f'   {tissue}: Chi2={chi2_t:.2f}, p={p_t:.2e} {"✅" if p_t < 0.05 else ""}')

# 3. 각 세포 유형별 Stage 간 비교
print('\n3️⃣ 세포 유형별 Stage 간 비율 비교:')
results_list = []

for cell_type in adata.obs['cell_type'].unique():
    row = {'Cell Type': cell_type}
    for stage in adata.obs[stage_col].dropna().unique():
        subset = adata[adata.obs[stage_col] == stage]
        ratio = (subset.obs['cell_type'] == cell_type).mean() * 100
        row[stage] = round(ratio, 2)
    results_list.append(row)

results_df = pd.DataFrame(results_list)
print(results_df.to_string(index=False))

# 결과 저장
results_df.to_csv(f'{OUTPUT_PATH}/step6_statistical_results.csv', index=False)

print(f'\n✅ STEP 6 완료!')

In [ ]:
# @title STEP 7: 종합 결과 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
stage_col = 'hbv_stage' if 'hbv_stage' in adata.obs.columns else 'Stage'

# Summary 생성
summary = f"""
================================================================================
🦠 HBV 면역세포 비교 분석 결과 (GSE182159 Real Data)
================================================================================

📅 분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

📊 데이터 요약:
   - 총 세포 수: {adata.n_obs:,}
   - 유전자 수: {adata.n_vars:,}
   - HBV Stage 수: {adata.obs[stage_col].nunique()}
   - 세포 유형 수: {adata.obs['cell_type'].nunique()}

📋 조직별 세포 수:
{adata.obs['tissue'].value_counts().to_string()}

📋 HBV Stage별 세포 수:
{adata.obs[stage_col].value_counts().to_string()}

🔬 통계 결과:
   - Chi-square (Cell Type × Stage): {chi2:.2f}
   - p-value: {p_value:.2e}
   - 결론: {'세포 분포가 HBV Stage에 따라 유의하게 다름' if p_value < 0.05 else '유의한 차이 없음'}

📁 저장된 파일:
   - step2_umap_overview.png
   - step2_umap_by_stage.png
   - step3_cell_type_distribution.png
   - step4_tissue_comparison.png
   - step5_exhaustion_markers.png
   - step6_statistical_results.csv
   - cell_type_ratios_by_stage.csv

================================================================================
"""

print(summary)

# Summary 저장
with open(f'{OUTPUT_PATH}/analysis_summary_{timestamp}.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print('\n🎉 모든 분석이 완료되었습니다!')
print(f'📁 결과 폴더: {OUTPUT_PATH}')

---
## 📚 결과 해석 가이드

### HBV Stage별 예상 면역 특성:

| Stage | CD8+ T | NK cells | PD-1 | 설명 |
|-------|--------|----------|------|------|
| IT (Immune Tolerant) | 낮음 | 낮음 | 높음 | 면역 관용, 바이러스 활발 복제 |
| IA (Immune Active) | 높음 | 높음 | 중간 | 면역 활성화, 간 손상 |
| IC (Immune Control) | 중간 | 중간 | 낮음 | 바이러스 억제, 안정기 |
| Resolved | 정상 | 정상 | 낮음 | 완치 상태 |

### 다음 분석 제안:
1. T세포 아형 세분화 분석
2. 간 상주 면역세포 vs 순환 면역세포 비교
3. 치료 반응 예측 바이오마커 발굴